In [ ]:
# # Tried it, didn't worked, move to something else
# # pb avec orateurs, etc.
# # mais semble confirmer les 242 paragraphes

# import pandas as pd

# # Chemin vers le fichier XML
# xml_path = "CRSANR5L16S2024O1N220.xml"

# # Lire tous les paragraphes comme lignes du DataFrame
# df_paragraphs = pd.read_xml(
#     xml_path,
#     xpath=".//ns:paragraphe",
#     namespaces={"ns": "http://schemas.assemblee-nationale.fr/referentiel"},
#     encoding="utf-8"
# )

# print(df_paragraphs.shape)

In [ ]:
# V1

# import xml.etree.ElementTree as ET
# import csv

# # Charger le fichier
# tree = ET.parse("CRSANR5L16S2024O1N220.xml")
# root = tree.getroot()
# ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}

# # Métadonnées globales
# uid = root.find(".//ns:uid", ns).text
# date_seance = root.find(".//ns:dateSeanceJour", ns).text
# president_elem = root.find(".//ns:presidentSeance", ns)
# president = president_elem.text if president_elem is not None else ""

# # Préparer lignes
# rows = []

# # Parcourir les <point> (chaque point = sujet)
# for point in root.findall(".//ns:point", ns):
#     sujet = point.find("ns:texte", ns)
#     sujet_texte = sujet.text.strip() if sujet is not None and sujet.text else ""
#     valeur_ptsodj = point.attrib.get("valeur_ptsodj", "")

#     # Parcourir les <paragraphe> de chaque point
#     for para in point.findall("ns:paragraphe", ns):
#         texte_elem = para.find("ns:texte", ns)
#         if texte_elem is None:
#             continue

#         # Texte brut et attribut stime
#         texte = "".join(texte_elem.itertext()).strip()
#         stime = texte_elem.attrib.get("stime", "")

#         # Orateur
#         orateur_elem = para.find(".//ns:orateur", ns)
#         nom, qualite, id_orateur = "", "", ""
#         if orateur_elem is not None:
#             nom = (
#                 orateur_elem.find("ns:nom", ns).text
#                 if orateur_elem.find("ns:nom", ns) is not None
#                 else ""
#             )
#             qualite = (
#                 orateur_elem.find("ns:qualite", ns).text
#                 if orateur_elem.find("ns:qualite", ns) is not None
#                 else ""
#             )
#             id_orateur = (
#                 orateur_elem.find("ns:id", ns).text
#                 if orateur_elem.find("ns:id", ns) is not None
#                 else ""
#             )

#         # Attributs de paragraphe
#         row = {
#             "UID": uid,
#             "Date séance": date_seance,
#             "Président": president,
#             "Sujet": sujet_texte,
#             "Valeur ODJ": valeur_ptsodj,
#             "ID paragraphe": para.attrib.get("id_syceron", ""),
#             "Ordre séance": para.attrib.get("ordre_absolu_seance", ""),
#             "Code grammaire": para.attrib.get("code_grammaire", ""),
#             "Code style": para.attrib.get("code_style", ""),
#             "Code parole": para.attrib.get("code_parole", ""),
#             "Role débat": para.attrib.get("roledebat", ""),
#             "Nom orateur": nom,
#             "Qualité orateur": qualite,
#             "ID orateur": id_orateur,
#             "Texte": texte,
#             "stime": stime,
#         }
#         rows.append(row)

# # Écrire en CSV
# with open("assemblee_debat_complet.csv", "w", newline="", encoding="utf-8") as f:
#     writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
#     writer.writeheader()
#     writer.writerows(rows)

# print("✅ Fichier CSV généré : assemblee_debat_complet.csv")


✅ Fichier CSV généré : assemblee_debat_complet.csv


In [ ]:
# ON MANQUAIT DES ÉLÉMENTS

# # V2 : récupérer plus de metadonnées pour le contexte
# # ça ça marche-ish (visiblement on rate quelques éléments ?)

# import xml.etree.ElementTree as ET
# import csv

# # Charger le fichier XML
# tree = ET.parse("CRSANR5L16S2024O1N220.xml")
# root = tree.getroot()
# ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}


# # Métadonnées générales du document
# def get_text(path):
#     elem = root.find(path, ns)
#     return elem.text.strip() if elem is not None and elem.text else ""


# metadata = {
#     "UID": get_text(".//ns:uid"),
#     "SeanceRef": get_text(".//ns:seanceRef"),
#     "SessionRef": get_text(".//ns:sessionRef"),
#     "DateSeance": get_text(".//ns:dateSeance"),
#     "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
#     "NumSeanceJour": get_text(".//ns:numSeanceJour"),
#     "NumSeance": get_text(".//ns:numSeance"),
#     "TypeAssemblee": get_text(".//ns:typeAssemblee"),
#     "Legislature": get_text(".//ns:legislature"),
#     "Session": get_text(".//ns:session"),
#     "NomFichierJO": get_text(".//ns:nomFichierJo"),
#     # "Validite": get_text(".//ns:validite"),
#     # "Etat": get_text(".//ns:etat"),
#     # "Diffusion": get_text(".//ns:diffusion"),
#     # "Version": get_text(".//ns:version"),
#     # "Environnement": get_text(".//ns:environnement"),
#     "President": get_text(".//ns:presidentSeance"),
# }

# # Extraire chaque paragraphe comme une ligne
# rows = []

# for point in root.findall(".//ns:point", ns):
#     sujet = point.find("ns:texte", ns)
#     sujet_texte = sujet.text.strip() if sujet is not None and sujet.text else ""
#     valeur_ptsodj = point.attrib.get("valeur_ptsodj", "")

#     for para in point.findall("ns:paragraphe", ns):
#         texte_elem = para.find("ns:texte", ns)
#         if texte_elem is None:
#             continue

#         texte = "".join(texte_elem.itertext()).strip()
#         stime = texte_elem.attrib.get("stime", "")

#         orateur_elem = para.find(".//ns:orateur", ns)
#         nom, qualite, id_orateur = "", "", ""
#         if orateur_elem is not None:
#             nom = (
#                 orateur_elem.find("ns:nom", ns).text
#                 if orateur_elem.find("ns:nom", ns) is not None
#                 else ""
#             )
#             qualite = (
#                 orateur_elem.find("ns:qualite", ns).text
#                 if orateur_elem.find("ns:qualite", ns) is not None
#                 else ""
#             )
#             id_orateur = (
#                 orateur_elem.find("ns:id", ns).text
#                 if orateur_elem.find("ns:id", ns) is not None
#                 else ""
#             )

#         # Attributs de paragraphe
#         row = {
#             **metadata,
#             "Sujet": sujet_texte,
#             "Valeur_ODJ": valeur_ptsodj,
#             "ID_paragraphe": para.attrib.get("id_syceron", ""),
#             "Ordre_seance": para.attrib.get("ordre_absolu_seance", ""),
#             "Code_grammaire": para.attrib.get("code_grammaire", ""),
#             "Code_style": para.attrib.get("code_style", ""),
#             "Code_parole": para.attrib.get("code_parole", ""),
#             "Role_debat": para.attrib.get("roledebat", ""),
#             "Nom_orateur": nom,
#             "Qualite_orateur": qualite,
#             "ID_orateur": id_orateur,
#             "stime": stime,
#             "Texte": texte,
#         }

#         rows.append(row)

# # Écriture CSV
# output_file = "V2.csv"
# with open(output_file, "w", newline="", encoding="utf-8") as f:
#     fieldnames = list(rows[0].keys())
#     writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
#     writer.writeheader()
#     writer.writerows(rows)

✅ Fichier CSV global généré : assemblee_debat_meta.csv


In [ ]:
# V3

# ça c'était mouaif, faisait merder le csv
# vérifier, mauvais nb col dans la lecture pandas après ?
# -> corrigé avec le quoting=csv.QUOTE_ALL
# voir si plus simple de faire un df pandas direct et export en csv ensuite ?

# Garder en tête : selon façon dont sauve le df etc. va changer le type des col
# et c'est pour ça que les ID paraph passent en float (car données manquantes ?)

import xml.etree.ElementTree as ET
import csv
import pandas as pd

# Charger le XML
tree = ET.parse("CRSANR5L16S2024O1N220.xml")
root = tree.getroot()
ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}


def get_text(path):
    elem = root.find(path, ns)
    return elem.text.strip() if elem is not None and elem.text else ""


# Métadonnées générales
metadata = {
    "UID": get_text(".//ns:uid"),
    "SeanceRef": get_text(".//ns:seanceRef"),
    "SessionRef": get_text(".//ns:sessionRef"),
    "DateSeance": get_text(".//ns:dateSeance"),
    "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
    "NumSeanceJour": get_text(".//ns:numSeanceJour"),
    "NumSeance": get_text(".//ns:numSeance"),
    "TypeAssemblee": get_text(".//ns:typeAssemblee"),
    "Legislature": get_text(".//ns:legislature"),
    "Session": get_text(".//ns:session"),
    "NomFichierJO": get_text(".//ns:nomFichierJo"),
    # "Validite": get_text(".//ns:validite"),
    # "Etat": get_text(".//ns:etat"),
    # "Diffusion": get_text(".//ns:diffusion"),
    # "Version": get_text(".//ns:version"),
    # "Environnement": get_text(".//ns:environnement"),
    "President": get_text(".//ns:presidentSeance"),
}

# Préparer les lignes du CSV
rows = []

# Contexte en cours
titre_general = ""
sous_titre = ""

# Parcours des éléments dans <contenu>
for elem in root.find("ns:contenu", ns).iter():
    tag = elem.tag.split("}")[-1]
    code_grammaire = elem.attrib.get("code_grammaire", "")
    code_style = elem.attrib.get("code_style", "")

    # Mémoriser le titre général si balise de titre
    if code_grammaire == "TITRE_TEXTE_DISCUSSION" and code_style == "Titre":
        texte = elem.find("ns:texte", ns)
        titre_general = texte.text.strip() if texte is not None and texte.text else ""

    # Mémoriser le sous-titre
    elif (
        code_grammaire == "SOUS_TITRE_TEXTE_DISCUSSION"
        and code_style == "Sous-tit_info"
    ):
        texte = elem.find("ns:texte", ns)
        sous_titre = texte.text.strip() if texte is not None and texte.text else ""

    # Si c’est un paragraphe, on enregistre une ligne
    elif tag == "paragraphe":
        texte_elem = elem.find("ns:texte", ns)
        if texte_elem is None:
            continue

        texte = "".join(texte_elem.itertext()).strip()
        stime = texte_elem.attrib.get("stime", "")

        orateur_elem = elem.find(".//ns:orateur", ns)
        nom, qualite, id_orateur = "", "", ""
        if orateur_elem is not None:
            nom = (
                orateur_elem.find("ns:nom", ns).text
                if orateur_elem.find("ns:nom", ns) is not None
                else ""
            )
            qualite = (
                orateur_elem.find("ns:qualite", ns).text
                if orateur_elem.find("ns:qualite", ns) is not None
                else ""
            )
            id_orateur = (
                orateur_elem.find("ns:id", ns).text
                if orateur_elem.find("ns:id", ns) is not None
                else ""
            )

        rows.append(
            {
                **metadata,
                "Titre_general": titre_general,
                "Sous_titre": sous_titre,
                "ID_paragraphe": elem.attrib.get("id_syceron", ""),
                "Ordre_seance": elem.attrib.get("ordre_absolu_seance", ""),
                "Code_grammaire": code_grammaire,
                "Code_style": code_style,
                "Code_parole": elem.attrib.get("code_parole", ""),
                "Role_debat": elem.attrib.get("roledebat", ""),
                "Nom_orateur": nom,
                "Qualite_orateur": qualite,
                "ID_orateur": id_orateur,
                "stime": stime,
                "Texte": texte,
            }
        )

# # Écriture CSV
# with open("assemblee_debat_meta_contexte.csv", "w", newline="", encoding="utf-8") as f:
#     fieldnames = list(rows[0].keys())
#     writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
#     writer.writeheader()
#     writer.writerows(rows)

df = pd.DataFrame(rows)
df.to_csv("V3.csv", index=False, quoting=csv.QUOTE_ALL)
print(df.shape)
# Garder en tête : selon façon dont sauve le df etc. va changer le type des col
# et c'est pour ça que les ID paraph passent en float (car données manquantes ?)


✅ Fichier CSV avec contexte généré : assemblee_debat_meta_contexte.csv


In [25]:
df_v3 = pd.read_csv("V3.csv")
print(df_v3.shape)

df_v3.head()

(242, 25)


,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Ordre_seance,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte
0,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,Session ordinaire 2023-2024,...,2,OUV_SEAN_2_1,NORMAL,NaN,president,Mme la présidente,NaN,719874.0,1063.52,La séance est ouverte.
1,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,Session ordinaire 2023-2024,...,3,OUV_SEAN_2_2,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est ouverte à vingt et une heures t...
2,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,Session ordinaire 2023-2024,...,7,ODJ_APPEL_DISCUSSION,NORMAL,NaN,president,Mme la présidente,NaN,719874.0,1067.56,L’ordre du jour appelle la suite de la discuss...
3,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,Session ordinaire 2023-2024,...,9,PAROLE_GENERIQUE,NORMAL,NaN,NaN,Mme la présidente,NaN,719874.0,1075.52,La parole est à M. le ministre délégué chargé ...
4,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,Session ordinaire 2023-2024,...,10,PAROLE_GENERIQUE,NORMAL,NaN,NaN,M. Frédéric Valletoux,ministre délégué chargé de la santé et de la p...,795350.0,1117.20,Le cancer touche chaque jour près de 1 200 per...


In [2]:
# V4
# Refactorisation claude
# Plus d'informations (contexte des points)
# Traçabilité (source d'extraction)

# Essai refactorisation claude
import pandas as pd
import xml.etree.ElementTree as ET
import csv
import os
import glob


def extraire_donnees_assemblee(fichier_xml: str) -> pd.DataFrame:
    """
    Extrait les données d'un fichier XML de compte rendu de l'Assemblée nationale
    Version hybride : combine contexte global ET informations des points ODJ.
    """

    # Charger le XML
    tree = ET.parse(fichier_xml)
    root = tree.getroot()
    ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}

    def get_text(path):
        """Fonction helper pour extraire le texte de manière sûre"""
        elem = root.find(path, ns)
        return elem.text.strip() if elem is not None and elem.text else ""

    # Métadonnées générales
    metadata = {
        "UID": get_text(".//ns:uid"),
        "SeanceRef": get_text(".//ns:seanceRef"),
        "SessionRef": get_text(".//ns:sessionRef"),
        "DateSeance": get_text(".//ns:dateSeance"),
        "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
        "NumSeanceJour": get_text(".//ns:numSeanceJour"),
        "NumSeance": get_text(".//ns:numSeance"),
        "TypeAssemblee": get_text(".//ns:typeAssemblee"),
        "Legislature": get_text(".//ns:legislature"),
        "Session": get_text(".//ns:session"),
        "NomFichierJO": get_text(".//ns:nomFichierJo"),
        "President": get_text(".//ns:presidentSeance"),
    }

    # ÉTAPE 1 : Créer le mapping paragraphe -> point (contexte local)
    paragraphe_to_point = _creer_mapping_points(root, ns)

    # ÉTAPE 2 : Parcourir le contenu et extraire tous les paragraphes
    contenu = root.find("ns:contenu", ns)
    if contenu is None:
        print(f"⚠️ Pas de contenu trouvé dans {fichier_xml}")
        return pd.DataFrame()

    rows = []
    contexte_global = _initialiser_contexte()

    for elem in contenu.iter():
        tag = elem.tag.split("}")[-1]
        code_grammaire = elem.attrib.get("code_grammaire", "")
        code_style = elem.attrib.get("code_style", "")

        # Mettre à jour le contexte global
        _mettre_a_jour_contexte(elem, contexte_global, ns)

        # Traiter les paragraphes
        if tag == "paragraphe":
            row = _extraire_paragraphe(
                elem, metadata, contexte_global, paragraphe_to_point, ns
            )
            if row:
                rows.append(row)

    return pd.DataFrame(rows)


def _creer_mapping_points(root, ns):
    """
    Crée un mapping entre les paragraphes et leurs points parents
    pour récupérer le contexte local (sujet du point, valeur ODJ).
    """
    paragraphe_to_point = {}

    for point in root.findall(".//ns:point", ns):
        # Informations du point
        sujet_elem = point.find("ns:texte", ns)
        sujet_texte = (
            sujet_elem.text.strip()
            if sujet_elem is not None and sujet_elem.text
            else ""
        )
        valeur_ptsodj = point.attrib.get("valeur_ptsodj", "")

        # Récupérer d'autres métadonnées du point si disponibles
        point_id = point.attrib.get("id", "")
        point_type = point.attrib.get("type", "")

        point_info = {
            "sujet": sujet_texte,
            "valeur_odj": valeur_ptsodj,
            "point_id": point_id,
            "point_type": point_type,
        }

        # Associer tous les paragraphes de ce point (recherche récursive)
        for para in point.findall(".//ns:paragraphe", ns):
            para_id = para.attrib.get("id_syceron", "")
            if para_id:  # Seulement si on a un ID valide
                paragraphe_to_point[para_id] = point_info
            else:
                # Fallback : utiliser l'objet Python comme clé
                paragraphe_to_point[id(para)] = point_info

    return paragraphe_to_point


def _initialiser_contexte():
    """Initialise la structure de contexte global"""
    return {
        "titre_general": "",
        "sous_titre": "",
        "contexte_stack": [],
        "section_courante": "",
    }


def _mettre_a_jour_contexte(elem, contexte, ns):
    """
    Met à jour le contexte global basé sur l'élément courant.
    Gère différents niveaux hiérarchiques de titres.
    """
    code_grammaire = elem.attrib.get("code_grammaire", "")
    code_style = elem.attrib.get("code_style", "")

    texte_elem = elem.find("ns:texte", ns)
    texte = (
        texte_elem.text.strip() if texte_elem is not None and texte_elem.text else ""
    )

    if not texte:
        return

    # Titre principal
    if code_grammaire == "TITRE_TEXTE_DISCUSSION" and code_style == "Titre":
        contexte["titre_general"] = texte
        contexte["contexte_stack"] = [texte]

    # Sous-titre
    elif (
        code_grammaire == "SOUS_TITRE_TEXTE_DISCUSSION"
        and code_style == "Sous-tit_info"
    ):
        contexte["sous_titre"] = texte
        if len(contexte["contexte_stack"]) == 1:
            contexte["contexte_stack"].append(texte)
        else:
            contexte["contexte_stack"] = [contexte["titre_general"], texte]

    # Autres types de sections
    elif code_grammaire in ["TITRE_RUBRIQUE", "SOUS_TITRE_RUBRIQUE"]:
        if "SOUS" in code_grammaire:
            if len(contexte["contexte_stack"]) >= 2:
                contexte["contexte_stack"][1] = texte
            else:
                contexte["contexte_stack"].append(texte)
        else:
            contexte["contexte_stack"] = [texte]

    # Section courante pour d'autres cas
    elif code_style in ["Titre", "Sous-tit", "Sous-tit_info"]:
        contexte["section_courante"] = texte


def _extraire_paragraphe(elem, metadata, contexte_global, paragraphe_to_point, ns):
    """
    Extrait toutes les informations d'un paragraphe.
    Combine contexte global et contexte local (point).
    """
    texte_elem = elem.find("ns:texte", ns)
    if texte_elem is None:
        return None

    # Extraction du texte avec gestion des balises internes
    texte = "".join(texte_elem.itertext()).strip()
    if not texte:  # Skip les paragraphes vides
        return None

    stime = texte_elem.attrib.get("stime", "")

    # Informations orateur
    nom, qualite, id_orateur = _extraire_orateur(elem, ns)

    # Récupérer le contexte du point parent (si disponible)
    para_id = elem.attrib.get("id_syceron", "")
    point_info = paragraphe_to_point.get(para_id) or paragraphe_to_point.get(
        id(elem), {}
    )

    # Déterminer la source d'extraction
    source_extraction = "point" if point_info else "global"

    # Construire le contexte hiérarchique complet
    contexte_hierarchique = " > ".join(filter(None, contexte_global["contexte_stack"]))

    return {
        **metadata,
        # Contexte global
        "Titre_general": contexte_global["titre_general"],
        "Sous_titre": contexte_global["sous_titre"],
        "Contexte_hierarchique": contexte_hierarchique,
        "Section_courante": contexte_global["section_courante"],
        # Contexte local du point
        "Sujet_point": point_info.get("sujet", ""),
        "Valeur_ODJ": point_info.get("valeur_odj", ""),
        "Point_ID": point_info.get("point_id", ""),
        "Point_type": point_info.get("point_type", ""),
        # Informations du paragraphe
        "ID_paragraphe": para_id,
        "Ordre_seance": elem.attrib.get("ordre_absolu_seance", ""),
        "Code_grammaire": elem.attrib.get("code_grammaire", ""),
        "Code_style": elem.attrib.get("code_style", ""),
        "Code_parole": elem.attrib.get("code_parole", ""),
        "Role_debat": elem.attrib.get("roledebat", ""),
        # Informations de l'orateur
        "Nom_orateur": nom,
        "Qualite_orateur": qualite,
        "ID_orateur": id_orateur,
        # Contenu
        "stime": stime,
        "Texte": texte,
        # Métadonnées d'extraction
        "Source_extraction": source_extraction,
        # nope "Fichier_source": os.path.basename(elem.getroottree().getroot().attrib.get("fichier", ""))
    }


def _extraire_orateur(elem, ns):
    """Extrait les informations de l'orateur de manière robuste"""
    orateur_elem = elem.find(".//ns:orateur", ns)
    nom, qualite, id_orateur = "", "", ""

    if orateur_elem is not None:
        nom_elem = orateur_elem.find("ns:nom", ns)
        nom = nom_elem.text.strip() if nom_elem is not None and nom_elem.text else ""

        qualite_elem = orateur_elem.find("ns:qualite", ns)
        qualite = (
            qualite_elem.text.strip()
            if qualite_elem is not None and qualite_elem.text
            else ""
        )

        id_elem = orateur_elem.find("ns:id", ns)
        id_orateur = (
            id_elem.text.strip() if id_elem is not None and id_elem.text else ""
        )

    return nom, qualite, id_orateur


def traiter_dossier_xml(dossier_path: str, pattern: str = "*.xml") -> pd.DataFrame:
    """
    Traite tous les fichiers XML d'un dossier et retourne un DataFrame consolidé.
    Avec gestion améliorée des erreurs et statistiques.
    """
    fichiers_xml = glob.glob(os.path.join(dossier_path, pattern))

    if not fichiers_xml:
        print(f"❌ Aucun fichier XML trouvé dans {dossier_path}")
        return pd.DataFrame()

    tous_dataframes = []
    stats = {"success": 0, "errors": 0, "total_rows": 0}

    print(f"🔄 Traitement de {len(fichiers_xml)} fichiers XML...")

    for fichier in fichiers_xml:
        nom_fichier = os.path.basename(fichier)
        print(f"   📄 {nom_fichier}...", end=" ")

        try:
            df = extraire_donnees_assemblee(fichier)
            if not df.empty:
                tous_dataframes.append(df)
                stats["success"] += 1
                stats["total_rows"] += len(df)
                print(f"✅ {len(df)} lignes")
            else:
                print("⚠️ Vide")

        except Exception as e:
            stats["errors"] += 1
            print(f"❌ Erreur: {str(e)}")

    print(f"\n📊 Statistiques:")
    print(f"   ✅ Fichiers traités avec succès: {stats['success']}")
    print(f"   ❌ Erreurs: {stats['errors']}")
    print(f"   📝 Total de lignes extraites: {stats['total_rows']}")

    if tous_dataframes:
        df_final = pd.concat(tous_dataframes, ignore_index=True)

        # Statistiques sur la répartition des sources
        if "Source_extraction" in df_final.columns:
            source_stats = df_final["Source_extraction"].value_counts()
            print(f"\n🎯 Répartition des sources:")
            for source, count in source_stats.items():
                print(
                    f"   {source}: {count} paragraphes ({count / len(df_final) * 100:.1f}%)"
                )

        return df_final
    else:
        return pd.DataFrame()


def analyser_extraction(df: pd.DataFrame):
    """
    Analyse les résultats de l'extraction pour identifier les différences.
    """
    if df.empty:
        print("❌ DataFrame vide")
        return

    print(f"\n📊 ANALYSE DE L'EXTRACTION")
    print(f"Total de paragraphes: {len(df)}")

    # Répartition par source
    if "Source_extraction" in df.columns:
        print(f"\n🔍 Répartition par source:")
        source_counts = df["Source_extraction"].value_counts()
        for source, count in source_counts.items():
            print(f"   {source}: {count} ({count / len(df) * 100:.1f}%)")

    # Paragraphes avec vs sans contexte de point
    if "Sujet_point" in df.columns:
        avec_point = df[df["Sujet_point"] != ""]
        sans_point = df[df["Sujet_point"] == ""]
        print(f"\n📝 Contexte des points:")
        print(f"   Avec sujet de point: {len(avec_point)}")
        print(f"   Sans sujet de point: {len(sans_point)}")

    # Types de codes grammaire les plus fréquents
    if "Code_grammaire" in df.columns:
        print(f"\n🏷️ Top 5 des codes grammaire:")
        top_codes = df["Code_grammaire"].value_counts().head()
        for code, count in top_codes.items():
            print(f"   {code or '(vide)'}: {count}")


# # Exemple d'utilisation
# if __name__ == "__main__":

#     # Pour un seul fichier
#     print("🚀 Test sur un fichier unique...")
#     df = extraire_donnees_assemblee("CRSANR5L16S2024O1N220.xml")

#     # Analyse des résultats
#     analyser_extraction(df)

#     # Sauvegarde
#     if not df.empty:
#         df.to_csv("assemblee_debat_hybride_ameliore.csv",
#                   index=False,
#                   quoting=csv.QUOTE_ALL,
#                   encoding='utf-8')
#         print(f"\n✅ Fichier CSV généré : assemblee_debat_hybride_ameliore.csv")

#     # # Pour un dossier entier (décommenter si besoin)
#     # df_complet = traiter_dossier_xml("./dossier_xml/")
#     # analyser_extraction(df_complet)

In [3]:
df_temp = extraire_donnees_assemblee("CRSANR5L16S2024O1N220.xml")
df_temp.to_csv("V4.csv")

df_v4 = pd.read_csv("V4.csv")
print(df_v4.shape)
df_v4.head()

(242, 33)


,Unnamed: 0,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,...,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction
0,0,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,...,OUV_SEAN_2_1,NORMAL,NaN,president,Mme la présidente,NaN,719874.0,1063.52,La séance est ouverte.,global
1,1,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,...,OUV_SEAN_2_2,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est ouverte à vingt et une heures t...,global
2,2,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,...,ODJ_APPEL_DISCUSSION,NORMAL,NaN,president,Mme la présidente,NaN,719874.0,1067.56,L’ordre du jour appelle la suite de la discuss...,point
3,3,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,...,PAROLE_GENERIQUE,NORMAL,NaN,NaN,Mme la présidente,NaN,719874.0,1075.52,La parole est à M. le ministre délégué chargé ...,point
4,4,CRSANR5L16S2024O1N220,RUANR5L16S2024IDS28385,SCR5A2024O1,20240530213000000,jeudi 30 mai 2024,3,220,AN,16,...,PAROLE_GENERIQUE,NORMAL,NaN,NaN,M. Frédéric Valletoux,ministre délégué chargé de la santé et de la p...,795350.0,1117.20,Le cancer touche chaque jour près de 1 200 per...,point


In [21]:
# V5:

# et dernier test refacto chat gpt

import xml.etree.ElementTree as ET
import pandas as pd
import csv

# Charger XML
tree = ET.parse("CRSANR5L16S2021O1N144.xml")
root = tree.getroot()
ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}


# Fonctions utilitaires
def get_text(path):
    elem = root.find(path, ns)
    return elem.text.strip() if elem is not None and elem.text else ""


def extract_orateur(elem):
    if elem is None:
        return "", "", ""
    nom = elem.findtext("ns:nom", default="", namespaces=ns)
    qualite = elem.findtext("ns:qualite", default="", namespaces=ns)
    id_orateur = elem.findtext("ns:id", default="", namespaces=ns)
    return nom, qualite, id_orateur


def clean_text(elem):
    return "".join(elem.itertext()).strip()


def build_metadata():
    return {
        "UID": get_text(".//ns:uid"),
        "SeanceRef": get_text(".//ns:seanceRef"),
        "SessionRef": get_text(".//ns:sessionRef"),
        "DateSeance": get_text(".//ns:dateSeance"),
        "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
        "NumSeanceJour": get_text(".//ns:numSeanceJour"),
        "NumSeance": get_text(".//ns:numSeance"),
        "TypeAssemblee": get_text(".//ns:typeAssemblee"),
        "Legislature": get_text(".//ns:legislature"),
        "Session": get_text(".//ns:session"),
        "NomFichierJO": get_text(".//ns:nomFichierJo"),
        "President": get_text(".//ns:presidentSeance"),
    }


def extract_paragraphs():
    metadata = build_metadata()
    rows = []
    titre_general = ""
    sous_titre = ""

    contenu = root.find("ns:contenu", ns)
    for elem in contenu.iter():
        tag = elem.tag.split("}")[-1]
        code_grammaire = elem.attrib.get("code_grammaire", "")
        code_style = elem.attrib.get("code_style", "")

        if code_grammaire == "TITRE_TEXTE_DISCUSSION" and code_style == "Titre":
            texte = elem.find("ns:texte", ns)
            titre_general = (
                texte.text.strip() if texte is not None and texte.text else ""
            )

        elif (
            code_grammaire == "SOUS_TITRE_TEXTE_DISCUSSION"
            and code_style == "Sous-tit_info"
        ):
            texte = elem.find("ns:texte", ns)
            sous_titre = texte.text.strip() if texte is not None and texte.text else ""

        elif tag == "paragraphe":
            texte_elem = elem.find("ns:texte", ns)
            if texte_elem is None:
                continue

            texte = clean_text(texte_elem)
            stime = texte_elem.attrib.get("stime", "")

            orateur_elem = elem.find(".//ns:orateur", ns)
            nom, qualite, id_orateur = extract_orateur(orateur_elem)

            row = {
                **metadata,
                "Titre_general": titre_general,
                "Sous_titre": sous_titre,
                "ID_paragraphe": elem.attrib.get("id_syceron", ""),
                "Ordre_seance": elem.attrib.get("ordre_absolu_seance", ""),
                "Code_grammaire": code_grammaire,
                "Code_style": code_style,
                "Code_parole": elem.attrib.get("code_parole", ""),
                "Role_debat": elem.attrib.get("roledebat", ""),
                "Nom_orateur": nom,
                "Qualite_orateur": qualite,
                "ID_orateur": id_orateur,
                "stime": stime,
                "Texte": texte,
            }
            rows.append(row)
    return rows


# Exécution
rows = extract_paragraphs()
df = pd.DataFrame(rows)
df.to_csv(
    "v5.csv",
    index=False,
    encoding="utf-8",
    quoting=csv.QUOTE_ALL,
)

In [24]:
df_v5 = pd.read_csv("V5.csv")
print(df_v5.shape)

df_v5.head()

(224, 25)


,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Ordre_seance,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte
0,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,2,OUV_SEAN_2_1,NORMAL,NaN,president,M. le président,NaN,606171.0,478.33,La séance est ouverte.
1,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,3,OUV_SEAN_2_2,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est ouverte à seize heures.)
2,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,7,ODJ_APPEL_DISCUSSION,NORMAL,NaN,president,M. le président,NaN,606171.0,481.57,L’ordre du jour appelle la discussion du proje...
3,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,9,PAROLE_GENERIQUE,NORMAL,PAROLE_1_1,NaN,M. le président,NaN,606171.0,491.77,La parole est à M. le ministre de l’intérieur.
4,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,10,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Gérald Darmanin,ministre de l’intérieur,607846.0,531.62,"Monsieur le président, mesdames, messieurs les..."


## Notes :
- virer les trucs inutiles (valide, prod, etc.) -> done
- voir si besoin ajouter comme info contexte les code_grammaire="TITRE_TEXTE_DISCUSSION" code_style="Titre" ou code_grammaire="SOUS_TITRE_TEXTE_DISCUSSION" code_style="Sous-tit_info" pour du contexte ? (= la thématique globale de la discussion ?) > testé. Mai sajoute des lignes (en gros moyen de les virer, pas l'air d'être un bug, jsute que y a des infos texte genre "l'amendement est retiré)
- qui ici sort pas puisque pas d'orateur (?), mais donc voir pour ajouter et contextualiser.
  
Pour chaque intervention, récupérer aussi id_syceron ?
- pour l'analyse, virer les prises de parole de Mme la présidente / le président (texte court et passage de parole)